# Segmenting and Clustering Neighborhoods in Toronto III
### Exercise 3


## Collect detils from web 

Obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

In [18]:
import pandas as pd

This logic is descibed in the previous notebook in detail.
* <a href="Segmenting and Clustering Neighborhoods in Toronto.ipynb">Previous Notebook</a>

It bring the data from the wikipedia page and the postal code from TORONTO. 

In [19]:

# No Need to run the following code, as it is loaded in picke file
if False:
    url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
    df = pd.read_html(url)[0] #pd.read_html(url, skiprows=1)[0]
    df.columns = df.iloc[0]
    df = df.iloc[1:]
    df.head()

    df = df[df['Borough'] != 'Not assigned']

    df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

    def row_handler(row):
        if row['Neighbourhood'] == 'Not assigned':
            return row['Borough'] 
        return row['Neighbourhood'] 

    df['Neighbourhood'] = df.apply(row_handler, axis=1)

    import numpy as np
    df['Latitude'] = np.nan
    df['Longitude'] = np.nan

    df.head()


#### My Struggle and the break-through
I struggled a lot with geocoder and I was using the google. The google map was failing all the time and the while loop go infinite. Then I realized that the issue is with the google api.
<br>
I then switched to <b> ArcGIS</b> provider and then it worked

<hr>
#### Tried the following methods but didnt work-out
<pre>

import requests # library to handle requests
geo_url = "https://geocoder.ca/?locate=101%20Wellington%20St,%20Ottawa&geoit=xml&json=1"
results = requests.get(url)
</pre>

In [20]:
#!conda install -c conda-forge geocoder
import geocoder # import geocoder

# No Need to run the following code, as it is loaded in picke file
if False:
    for index, row in df.iterrows():
        # initialize your variable to None
        lat_lng_coords = None

        # loop until you get the coordinates
        while(lat_lng_coords is None):
          g = geocoder.arcgis('{}, Toronto, Ontario'.format(row['Postcode']))
          lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        df.at[index, 'Latitude'] = latitude
        df.at[index, 'Longitude'] = longitude
        print(longitude,latitude, row['Postcode'] )


In [4]:
#df.head()

As I have got the pandas frame ready, I save this in a picke file so that I can get it later as-is, in order to reduce the call and to save the time 

In [21]:
# No Need to run the following code, as it is loaded in picke file
if False:
    df.to_pickle("./dummy.pkl")

In [22]:
df = pd.read_pickle("./dummy.pkl")

In [23]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [24]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')

Folium installed


This is old commented logic: donnt work
<!--
geolocator = Nominatim(user_agent="coursera-course.com")
location =  None
for index, row in df.iterrows():

    while(location is None):
        try:
            location = geolocator.geocode(row['Postcode'])     
        except:
            
    row['Latitude'] = location.latitude
    row['Longitude'] = location.longitude
    print(row['Postcode'], row['Latitude'] ,  row['Longitude'])
    location = None    
df.head()
-->

<!--
import geocoder # import geocoder

for index, row in df.iterrows():

    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(row['Postcode']))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df.at[index, 'Latitude'] = latitude
    df.at[index, 'Longitude'] = longitude
    
df.head()
-->

In [25]:
df['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [26]:
import geocoder
g = geocoder.arcgis('Toronto, ON')
latitude = g.latlng[0]
longitude = g.latlng[1]
print(latitude, longitude)

43.648690000000045 -79.38543999999996


In [27]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

map_address = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_address)  
    
map_address

In [11]:
CLIENT_ID = '4CTAG1JEDGEOAVDVG4KSAXTXXTHSVHQ35QK2NQHSF4FXSAYK' # your Foursquare ID
CLIENT_SECRET = 'V5ZEKZ52E2SH3CMKW3J5FW0PIJJO1KK1HTURDSR30QUJRM35' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4CTAG1JEDGEOAVDVG4KSAXTXXTHSVHQ35QK2NQHSF4FXSAYK
CLIENT_SECRET:V5ZEKZ52E2SH3CMKW3J5FW0PIJJO1KK1HTURDSR30QUJRM35


In [17]:
df.groupby('Borough').count()

,Postcode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,9,9,9,9
Downtown Toronto,18,18,18,18
East Toronto,5,5,5,5
East York,5,5,5,5
Etobicoke,12,12,12,12
Mississauga,1,1,1,1
North York,24,24,24,24
Queen's Park,1,1,1,1
Scarborough,17,17,17,17


In [32]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(38, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295410
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667985,-79.314642
3,M4M,East Toronto,Studio District,43.662766,-79.334830
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090


In [34]:
toronto_data['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [ ]:
# one hot encoding
df_onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [36]:
toronto_data.loc[0, 'Neighbourhood']

'The Beaches'

In [37]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67653131600008, -79.29540999999995.


In [38]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)



In [40]:
import requests # library to handle requests
results = requests.get(url).json()

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [43]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.678798,-79.298045
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
# type your answer here
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )




The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [47]:
print(toronto_venues.shape)
toronto_venues.head()

(1726, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.29541,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676531,-79.29541,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676531,-79.29541,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676531,-79.29541,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.683262,-79.35512,Dairy Queen,43.684223,-79.357062,Ice Cream Shop


In [49]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,60,60,60,60,60,60
"Brockton, Exhibition Place, Parkdale Village",54,54,54,54,54,54
Business reply mail Processing Centre969 Eastern,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",67,67,67,67,67,67
"Cabbagetown, St. James Town",41,41,41,41,41,41
Central Bay Street,93,93,93,93,93,93
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,9,9,9,9,9,9


In [52]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 212 uniques categories.


In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,...,Thrift / Vintage Store,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.010000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.016667,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.018519,0.000000,0.00,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business reply mail Processing Centre969 Eastern,0.000000,0.00,0.000000,0.000000,0.040000,0.00,0.00,0.010000,0.000000,...,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.014925,0.00,0.000000,0.000000,0.014925,0.00,0.00,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.00,0.000000,0.000000,0.010753,0.00,0.00,0.010753,0.010753,...,0.000000,0.010753,0.00,0.010753,0.000000,0.010753,0.010753,0.010753,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.01,0.030000,0.000000,...,0.010000,0.010000,0.00,0.050000,0.000000,0.000000,0.040000,0.010000,0.000000,0.000000
8,Christie,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.00,0.011905,0.011905,0.011905,0.00,0.00,0.000000,0.000000,...,0.000000,0.000000,0.00,0.011905,0.000000,0.011905,0.011905,0.000000,0.000000,0.000000


In [59]:
toronto_grouped.shape

(38, 212)

In [60]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.07
2                Hotel  0.06
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.05
2          Restaurant  0.05
3      Farmers Market  0.03
4  Seafood Restaurant  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1              Restaurant  0.06
2  Furniture / Home Store  0.06
3      Italian Restaurant  0.04
4                   Hotel  0.04


----Business reply mail Processing Centre969 Eastern----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.05
3           Restaurant  0.04
4  American Restaurant  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
      

In [61]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,American Restaurant,Gastropub,Japanese Restaurant,Burger Joint,Bar,Deli / Bodega
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Seafood Restaurant,Hotel,Café,Cheese Shop,Farmers Market,Beer Bar
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Sandwich Place,Café,Bar,Italian Restaurant,Gift Shop,Vegetarian / Vegan Restaurant,Hotel
3,Business reply mail Processing Centre969 Eastern,Coffee Shop,Bar,Café,American Restaurant,Steakhouse,Restaurant,Hotel,Italian Restaurant,Pub,Thai Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Italian Restaurant,Coffee Shop,Bar,Restaurant,Café,Pub,Gym / Fitness Center,Speakeasy,Sandwich Place,Bakery
5,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pizza Place,Café,Bakery,Italian Restaurant,Indian Restaurant,Park,Jewelry Store,Beer Store
6,Central Bay Street,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Tea Room,Plaza,Bubble Tea Shop,Japanese Restaurant,Bar,Sandwich Place
7,"Chinatown, Grange Park, Kensington Market",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Mexican Restaurant,Bakery,Art Gallery,Dumpling Restaurant,Coffee Shop
8,Christie,Café,Grocery Store,Baby Store,Playground,Italian Restaurant,Coffee Shop,Dumpling Restaurant,Fast Food Restaurant,Farmers Market,Farm
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Restaurant,Burger Joint,Sushi Restaurant,Dance Studio,Fast Food Restaurant,Men's Store,Bubble Tea Shop


## Cluster Neighborhoods

In [69]:
# set number of clusters

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       2, 0, 4, 4, 0, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [75]:
toronto_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M4E,East Toronto,The Beaches,43.676531,-79.295410,0
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667985,-79.314642,0
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090,0


In [74]:
toronto_data = toronto_data.rename(columns={'Neighbourhood': 'Neighborhood'})


In [76]:
toronto_merged = toronto_data

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295410,0,Other Great Outdoors,Pub,Coffee Shop,Wings Joint,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit
1,M4K,East Toronto,"The Danforth West, Riverdale",43.683262,-79.355120,0,Park,Grocery Store,Ice Cream Shop,Bus Line,Discount Store,Wings Joint,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Farm
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.667985,-79.314642,0,Liquor Store,Ice Cream Shop,Burger Joint,Fast Food Restaurant,Fish & Chips Shop,Sandwich Place,Steakhouse,Sushi Restaurant,Pub,Intersection
3,M4M,East Toronto,Studio District,43.662766,-79.334830,0,Diner,Italian Restaurant,Pizza Place,Bakery,Gastropub,American Restaurant,Coffee Shop,Bar,Arts & Crafts Store,Sushi Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728135,-79.387090,0,Bus Line,Swim School,Wings Joint,Eastern European Restaurant,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Exhibit


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Thanks a lot for visiting this notebook... <b>have a great day
    <hr>
    <b>Sharafudheen